In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import time

In [70]:
browser = webdriver.Chrome('C:\chromedriver.exe')
browser.get('https://www.bigkinds.or.kr/v2/news/index.do')

In [71]:
query_txt = input('크롤링할 키워드는 무엇입니까?:')
# 사용자에게 검색어 입력

element = browser.find_element_by_id("total-search-key")
element.send_keys(query_txt)  #검색어입력하게 하는 부분
element.send_keys("\n")  #엔터효과
#검색창의 이름을 찾아서 검색어를 입력

크롤링할 키워드는 무엇입니까?:넥슨


In [72]:
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

In [73]:
# 다음 페이지 넘기기

nextbtn = browser.find_element_by_xpath('//*[@id="news-results-tab"]/div[6]/div[1]/div/div/div/div[4]/a')
type(nextbtn)

selenium.webdriver.remote.webelement.WebElement

In [74]:
wholes = soup.select('div.news-inner')
len(wholes)

10

In [75]:
# [제목, 언론사, 분류, 날짜, 기자이름, 기사내용]

contents = [] 
for whole in wholes:
    title = whole.select('span.title-elipsis')
    press =  whole.select('div.info > div > a')
    category = whole.select('div.info > div > span.bullet-keyword')
    date = whole.select('div.info > p')
    contents.append([title[0].text.strip(), press[0].text.strip(), category[0].text.strip(), date[0].text.strip(), date_name[1].text.strip()])

len(contents)

10

In [79]:
contents[1]

['채용 설명회에 지역명소 구현까지... 가상을 일상으로 만든 메타버스',
 '아주경제',
 'IT_과학>콘텐츠',
 '2021/09/05',
 '정명섭']

In [76]:
# 기사 전문 

# texts = []
# for i in range(0, 100):
#     browser.find_elements_by_css_selector('span.title-elipsis')[i].click()
#     time.sleep(1)
#     text = browser.find_elements_by_css_selector('div.news-view-body')[0].text
#     texts.append(text.replace('\n', ''))
#     browser.find_element_by_xpath("//div[@id='news-detail-modal']/div/div/button").click()

# len(texts)

In [77]:
df = pd.DataFrame(data=contents, columns =['title', 'press', 'category', 'date', 'name'])